# Create an Amazon Bedrock Knowledge Base

This notebook demonstrates how to create and configure an Amazon Bedrock Knowledge Base. 

The Knowledge Base integrates Amazon S3 as the data source for onboarding documents and uses Amazon OpenSearch Serverless as the vector store. It enables retrieval-augmented generation (RAG) by powering intelligent queries over structured and unstructured HR content like onboarding guides, policies, and FAQs.

If you want to create the Knowledge Base with your own documents, change the documents uploaded in the **Create the Amazon S3 bucket and upload the sample documents** section. This component will be used in the next notebook of this folder.

## Setup and prerequisites

### Prerequisites
* Complete the prerequisites notebook located in this folder
* Python 3.10+
* AWS account
* Amazon Nova Micro enabled on Amazon Bedrock
* IAM role with permissions to create Amazon Bedrock Knowledge Base, Amazon S3 bucket, Amazon OpenSearch Serverless

Let's now install the requirement packages and define the needed clients to create our Amazon Bedrock Knowledge Base:

In [ ]:
%pip install -r requirements.txt

In [ ]:
import os
import json
import time
import uuid
import boto3
import logging
import requests
from datetime import datetime

In [ ]:
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime")

In [ ]:
# Get the current timestamp
current_time = time.time()
# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

## Download Amazon Bedrock Knowledge Bases helper
To expedite the knowledge base configuration and creation we will be downloading the knowledge base utility file. This contains a helper to generate knowledge bases abstracting the multiple API calls that need to be used.

In [ ]:
url = "https://raw.githubusercontent.com/aws-samples/amazon-bedrock-samples/main/rag/knowledge-bases/features-examples/utils/knowledge_base.py"
target_path = "utils/knowledge_base.py"
os.makedirs(os.path.dirname(target_path), exist_ok=True)
response = requests.get(url)
with open(target_path, "w") as f:
    f.write(response.text)
print(f"Downloaded Knowledge Bases utils to {target_path}")

In [ ]:
from utils.knowledge_base import BedrockKnowledgeBase

## Create Amazon Bedrock Knowledge Base
In this section we will configure the Amazon Bedrock Knowledge Base containing the policy documents andn FAQs for employee onboarding. We will be using Amazon Opensearch Serverless Service as the underlying vector store and Amazon S3 as the data source containing the files.

In [ ]:
knowledge_base_name = f"hr-agent-knowledge-base-{suffix}"
knowledge_base_description = "HR Agent Knowledge Base containing onboarding and benefits documentation."
foundation_model = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

For this notebook, we'll create a Knowledge Base with an Amazon S3 data source.

In [ ]:
data_bucket_name = f'bedrock-hr-agent-{suffix}-bucket' # replace it with your first bucket name.
data_sources=[{"type": "S3", "bucket_name": data_bucket_name}]

### Create the Amazon S3 bucket and upload the sample documents
For this notebook, we'll create a Knowledge Base with an Amazon S3 data source.

In [ ]:
import botocore
import os

def create_s3_bucket(bucket_name, region=None):
    s3 = boto3.client('s3', region_name=region)

    try:
        if region is None or region == 'us-east-1':
            s3.create_bucket(Bucket=bucket_name)
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"✅ Bucket '{bucket_name}' created successfully.")
    except botocore.exceptions.ClientError as e:
        print(f"❌ Failed to create bucket: {e.response['Error']['Message']}")

create_s3_bucket(data_bucket_name, region)


In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

In [ ]:
upload_directory("./onboarding_files", data_bucket_name)

### Create the Knowledge Base
We are now going to create the Knowledge Base using the abstraction located in the helper function we previously downloaded.

In [ ]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

### Start ingestion job
Once the KB and data source created, we can start the ingestion job for the data source. During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case OSS.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()

### Test the Knowledge Base
We can now test the Knowledge Base to verify the documents have been ingested properly.

In [ ]:
query = "Who is the medical insurance provider name?"

In [ ]:
foundation_model = "amazon.nova-micro-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

### Store the Knowledge Base ID
Store the ID of the generated Knowledge Base to use it in other notebooks of the repository.

In [ ]:
kb_region = region
%store kb_id
%store kb_region

## Clean up the resources
**If you plan to use other notebooks from this folder, do not delete yet the Knowledge Base as it will be needed.**

When you are finished with the other notebooks, to avoid additional costs, delete the resources created.

In [ ]:
'''
print("===============================Deleting Knowledge Base and associated resources==============================\n")
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)
'''